# Similar cities to Barueri in Sorocaba region
#### Rodrigo Medeiros

## Introduction

A retired couple which lives in Barueri desire to move to Sorocaba region to be close to their family, they don't know very well the region and have concerns about the city they are going to choose. They intend to move to a city similar to Barueri in terms of social development, demographic density, availability of restaurants, parks, museums and so on.

To verify the cities in Sorocaba region that are similar to Barueri in this aspects, will be used technics of web scrapping data and machine learning to identify and propose cities with best odds to satisfy the couple.

## Data

In this project will be used the Barueri page on Wikipédia (https://pt.wikipedia.org/wiki/Barueri) to retrieve follow information:
 - Area;
 - Population;
 - Human Development Index;

Also will be used used the Metropolitan Region of Sorocaba page on Wikipédia (https://pt.wikipedia.org/wiki/Regi%C3%A3o_Metropolitana_de_Sorocaba) to retrieve the same information for cities in this area.

The data about the venues (restaurants, parks, museums and etc.) for each city will be retrieve from Foursquare API considering a area of 7Km around the coordinates of the city.